In [ ]:
%cd C:/Users/Mathieu/Desktop/Projets/Benter

In [34]:
import datetime as dt
import numpy as np
import pandas as pd
import os

from scripts.scrape_pmu_data import get_pmu_api_url, execute_get_query,create_day_folder
from scripts.generate_pmu_data import get_num_pmu_enjeu, convert_queried_data_to_race_horse_df

import utils
from utils import features
from utils import import_data
from utils import model as utils_model
from utils import wagering_stategy
from constants import PMU_BETTINGS, PMU_DATA_DIR

def execute_queries(seconds_before:int, race_times:dict)->int:
    query_count=0
    prefix=f"{seconds_before}s_before"
    candidates = {(date, r_i, c_i):time_to_race for (date, r_i, c_i), time_to_race in race_times.items() if time_to_race.total_seconds()<seconds_before}
    for (date_, r_i, c_i), _ in candidates.items():
        for url_name in ["CITATIONS_INTERNET", "COMBINAISONS_INTERNET", "CITATIONS",
                     "COMBINAISONS",  ]:
            day_folder_path =os.path.join(PMU_DATA_DIR, date_.isoformat())
            filename = os.path.join(day_folder_path, f'{prefix}_R{r_i}_C{c_i}_{url_name.lower()}.json')
            if not os.path.exists(filename):
                url = get_pmu_api_url(url_name=url_name, r_i=r_i, c_i=c_i, date=date)
                utils.dump_json(
                    data=execute_get_query(url=url),
                    filename=filename)
                query_count += 1
    return query_count

TIMEZONE = 'Europe/Paris'
import pytz
tz = pytz.timezone(TIMEZONE)

for date in [dt.date.today(), dt.date.today()+dt.timedelta(days=1), dt.date.today()-dt.timedelta(days=1)]:
    programme = execute_get_query(url=get_pmu_api_url(url_name="PROGRAMME", date=date))
    date_ = dt.date.fromtimestamp(programme['programme']['date']/1000)
    create_day_folder(date=date_)
    

def update():

    dt_now = tz.localize(dt.datetime.now())

    race_times = {}
    for date in [dt.date.today(), dt.date.today()+dt.timedelta(days=1), dt.date.today()-dt.timedelta(days=1)]:
        programme = execute_get_query(url=get_pmu_api_url(url_name="PROGRAMME", date=date))
        for reunion in programme['programme']['reunions']:
            for course in reunion['courses']:
                date_ = dt.date.fromtimestamp(programme['programme']['date']/1000)
                race_times[(date_, course["numReunion"], course["numOrdre"])]=dt.datetime.fromtimestamp(
                            course["heureDepart"] / 1000,
                            tz=dt.timezone(dt.timedelta(milliseconds=course["timezoneOffset"])),
                        )-dt_now

    coming_races = {(date, r_i, c_i):time_to_race for (date, r_i, c_i), time_to_race in race_times.items() if time_to_race.total_seconds()>0}

    query_count=0
    # 5s
    query_count+=execute_queries(seconds_before=5,race_times=coming_races )


    # 30s
    query_count+=execute_queries(seconds_before=30,race_times=coming_races )

    # 1min
    query_count+=execute_queries(seconds_before=60,race_times=coming_races )

    return query_count

In [39]:
query_count=0
while True:
    start_date=dt.datetime.now()
    new_queries= update()
    end_date =dt.datetime.now()
    if new_queries:
        print(f"[{end_date.isoformat()}] Time to execute: {(end_date-start_date).total_seconds():.2}, {new_queries} queries made")
        query_count+=new_queries
    print(f"[{end_date.isoformat()}] Time to execute: {(end_date-start_date).total_seconds():.2}s, {query_count} queries in total", end="\r")

KeyboardInterrupt: 

In [ ]:
next_date, r_i, c_i = min(coming_races, key=coming_races.get)
print(f"Time to next race: {coming_races[(next_date, r_i, c_i)]}, date {date}, R{r_i} C{c_i}")

courses_ = [course for reunion in programme['programme']['reunions'] for course in reunion['courses'] if course["numReunion"]==r_i and course["numOrdre"]==c_i]
assert len(courses_)==1
course=courses_[0]

combinaisons =  execute_get_query(url=get_pmu_api_url(url_name="COMBINAISONS", date=date, r_i=r_i, c_i=c_i)+"?specialisation=INTERNET")
citations =  execute_get_query(url=get_pmu_api_url(url_name="CITATIONS", date=date, r_i=r_i, c_i=c_i)+"?specialisation=INTERNET")

for url_name in ["CITATIONS_INTERNET", "COMBINAISONS_INTERNET", "CITATIONS",
                 "COMBINAISONS",  ]:
    filename = os.path.join(day_folder_path, f'R{r_i}_C{c_i}_{url_name.lower()}.json')
    url = get_pmu_api_url(url_name=url_name, r_i=r_i, c_i=c_i, date=date)
    if replace_if_exists or not os.path.exists(filename) or not _check_query_json(filename=filename,
                                                                                  url=url):
        utils.dump_json(
            data=execute_get_query(url=url),
            filename=filename)
        query_count += 1